In [ ]:
! pip install -q youtube-transcript-api

In [ ]:
from langchain.document_loaders import YoutubeLoader

loader = YoutubeLoader.from_youtube_url(
    "https://www.youtube.com/watch?v=rVbvcIFeLYw&t=17s", add_video_info=False)


In [ ]:
transcript_docs = loader.load()

In [ ]:
from langchain.embeddings import LlamaCppEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.llms import LlamaCpp
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA

In [ ]:
embeddings = LlamaCppEmbeddings(
    model_path="[PATH_TO_YOUR_MODEL]/llama.cpp/models/llama-2-7b-chat/ggml-model-q5_0.gguf",
    verbose=False,
    )

text_spliter = RecursiveCharacterTextSplitter(
    chunk_size=250,
    chunk_overlap=25,
    length_function=len,
    add_start_index=True
)

documents = text_spliter.split_documents(transcript_docs)
vectorstore = Chroma.from_documents(documents, embeddings)


In [ ]:
llm  = LlamaCpp(
    model_path="[PATH_TO_YOUR_MODEL]/llama.cpp/models/llama-2-7b-chat/ggml-model-q5_0.gguf",
    verbose=False,
    temperature=0,
    n_gpu_layers=2,
    n_batch=512,
    n_ctx=2048,
    f16_kv=True
)

prompt_template = """
[INST]<<SYS>> You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.<</SYS>> 
Question: {question} 
Context: {context} 
Answer: [/INST]
"""

prompt = PromptTemplate.from_template(prompt_template)

qa_chain = RetrievalQA.from_chain_type(
    llm, 
    retriever=vectorstore.as_retriever(), 
    chain_type_kwargs={"prompt": prompt},
)



In [ ]:
question = "What are the distruption in AI?"
result = qa_chain({"query": question})
result["result"]

In [ ]:
question = "What are the list of distruption in AI? I mean distruption not disturbances."
result = qa_chain({"query": question})
result["result"]